### Setup

In [1]:
# install if not yet done
using Pkg

# Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
# Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

    Updating registry at `C:\Users\domin\.julia\registries\General`
┌ Info: The General registry is installed via git. Consider reinstalling it via
│ the newer faster direct from tarball format by running:
│   pkg> registry rm General; registry add General
│ 
└ @ Pkg.Registry C:\Users\domin\.julia\juliaup\julia-1.11.3+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Pkg\src\Registry\Registry.jl:478
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`
    Updating git-repo `https://github.com/bergermann/Dragoon.jl.git`
   Installed ArrayInterface ──── v7.19.0
   Installed ImageSegmentation ─ v1.9.0
   Installed libpng_jll ──────── v1.6.48+0
   Installed Unitful ─────────── v1.22.1
    Updating `C:\Users\domin\.julia\environments\v1.11\Project.toml`
  [2528b1d8] ~ Dragoon v0.5.0 `https://github.com/bergermann/Dragoon.jl.git#main` ⇒ v0.5.0 `https://github.com/bergermann/Dragoon.jl.git#main`
    Updating `C:\U

In [2]:
using Dragoon, Plots #, Dates

In [28]:
n = 20 # number of discs in the booster

initdist = findpeak1d(22.025e9,n) # initial disc configuration

freqs = genFreqs(22.025e9,50e6; n=10) # optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; n=1000) # plot with these frequencies

booster = AnalyticalBooster(initdist; ndisk=n)  # physical properties of the booster

hist = initHist(booster,10000,freqs,ObjAnalytical); # stores measurements
#                           ^ length, should be at least n

10000-element Vector{State}:
 State([0.007209715011297336, 0.01541943002259467, 0.023629145033892004, 0.03183886004518934, 0.04004857505648668, 0.04825829006778401, 0.05646800507908135, 0.06467772009037868, 0.07288743510167603, 0.08109715011297336, 0.08930686512427068, 0.097516580135568, 0.10572629514686534, 0.11393601015816267, 0.12214572516946, 0.13035544018075734, 0.13856515519205465, 0.14677487020335206, 0.15498458521464936, 0.1631943002259467], -243.30220712543468, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0,

### Optimizer

In [29]:
trace = simulatedAnnealing(booster,hist,freqs,
        100e-6,
        TempLinear(100,10001),
        ObjAnalytical,
        UnstuckDont;
        maxiter=Int(10001),
        nreset=500,
        nresetterm=10,
        showtrace=true,
        showevery=1000,
        unstuckisiter=true,
        traceevery=100);

Iter: 1000, timestamp: 5 minutes, 12 seconds
Iter finished. Objective value current:  -14530.654
               Objective value solution: -14530.654
               Temperature:              90.001

Iter: 2000, timestamp: 10 minutes, 30 seconds
Iter finished. Objective value current:  -14393.708
               Objective value solution: -14674.593
               Temperature:              80.002

Resetting to best solution.
Resetting to best solution.
Iter: 3000, timestamp: 15 minutes, 55 seconds
Iter finished. Objective value current:  -14621.093
               Objective value solution: -14694.233
               Temperature:              70.003

Resetting to best solution.
Resetting to best solution.
Iter: 4000, timestamp: 21 minutes, 19 seconds
Iter finished. Objective value current:  -14521.899
               Objective value solution: -14694.233
               Temperature:              60.004

Resetting to best solution.
Resetting to best solution.
Iter: 5000, timestamp: 26 minutes, 43

In [ ]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

In [ ]:
#get handy output
analyse(hist,trace[1:end-1],freqsplot; freqs=freqs,div=20,scale=1e9,ylim=[-0.05e4,3e4])

#savefig(plt[i],"cool_epic_result_wow.svg")

In [ ]:
booster.timestamp